In [1]:
import psutil
import numpy as np
import time
import logging
from sklearn.ensemble import IsolationForest

# Configure logging to write to both console and a log file
logging.basicConfig(
    format='%(asctime)s - %(message)s',
    level=logging.INFO,
    handlers=[
        logging.StreamHandler(),
        logging.FileHandler("threatDetection.log", mode='a')
    ]
)

# Initialize the Isolation Forest model
clf = IsolationForest(random_state=42)

# Function to collect current system resource usage
def get_system_resource_usage():
    cpu_usage = psutil.cpu_percent(interval=1)
    memory_usage = psutil.virtual_memory().percent
    return np.array([cpu_usage, memory_usage])

# Monitor CPU, Memory, Disk Usage
def monitor_system():
    current_usage = get_system_resource_usage()
    logging.info("Current System Resource Usage: CPU: %.2f%%, Memory: %.2f%%", current_usage[0], current_usage[1])
    return current_usage

# Function to detect suspicious processes
def detect_suspicious_processes():
    suspicious_processes = []
    for proc in psutil.process_iter(['pid', 'name', 'cpu_percent', 'memory_percent']):
        try:
            if proc.info['cpu_percent'] > 80 or proc.info['memory_percent'] > 50:
                suspicious_processes.append(proc.info)
        except (psutil.NoSuchProcess, psutil.AccessDenied):
            continue
    return suspicious_processes

# Monitor suspicious activity
def monitor_suspicious_activity():
    resource_data = []  # Collect resource usage data for anomaly detection

    while True:
        # Monitor system resources
        current_usage = monitor_system()
        resource_data.append(current_usage)

        # Check for suspicious processes
        suspicious_processes = detect_suspicious_processes()
        if suspicious_processes:
            logging.info("Suspicious Process Detected:")
            for proc in suspicious_processes:
                logging.info(f"PID: {proc['pid']}, Name: {proc['name']}, CPU: {proc['cpu_percent']}%, Memory: {proc['memory_percent']}%")

        # Train Isolation Forest on the collected resource data
        if len(resource_data) >= 5:  # Train after collecting a certain amount of data
            clf.fit(resource_data)
            # Predict anomalies in the most recent data point
            prediction = clf.predict([resource_data[-1]])
            if prediction == -1:
                logging.info("Potential threat detected in current system resource usage: %s", resource_data[-1])

        time.sleep(2)  # Monitor every 2 seconds

if __name__ == "__main__":
    try:
        monitor_suspicious_activity()
    except KeyboardInterrupt:
        logging.info("Monitoring stopped by user. System data saved to 'cybersecurity.log'.")


2024-10-01 15:08:33,540 - Current System Resource Usage: CPU: 2.30%, Memory: 75.00%
2024-10-01 15:08:39,472 - Current System Resource Usage: CPU: 0.70%, Memory: 75.10%
2024-10-01 15:08:40,273 - Suspicious Process Detected:
2024-10-01 15:08:40,274 - PID: 0, Name: System Idle Process, CPU: 1550.9%, Memory: 4.866997135772186e-05%
2024-10-01 15:08:43,290 - Current System Resource Usage: CPU: 2.90%, Memory: 74.90%
2024-10-01 15:08:44,070 - Suspicious Process Detected:
2024-10-01 15:08:44,071 - PID: 0, Name: System Idle Process, CPU: 1543.7%, Memory: 4.866997135772186e-05%
2024-10-01 15:08:47,084 - Current System Resource Usage: CPU: 4.90%, Memory: 74.40%
2024-10-01 15:08:47,900 - Suspicious Process Detected:
2024-10-01 15:08:47,901 - PID: 0, Name: System Idle Process, CPU: 1516.0%, Memory: 4.866997135772186e-05%
2024-10-01 15:08:50,926 - Current System Resource Usage: CPU: 0.70%, Memory: 74.50%
2024-10-01 15:08:51,664 - Suspicious Process Detected:
2024-10-01 15:08:51,665 - PID: 0, Name: Sy